In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [3]:
# Load dataset
file_path = "data/titanic.csv"  # Update this path if necessary
df = pd.read_csv(file_path)
pd.set_option('display.max_columns', None)
df.head()

,Passenger Class,Female,Male,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Survived
0,3,0,1,22.0,1,0,7.2500,0
1,1,1,0,38.0,1,0,71.2833,1
2,3,1,0,26.0,0,0,7.9250,1
3,1,1,0,35.0,1,0,53.1000,1
4,3,0,1,35.0,0,0,8.0500,0


In [4]:
# Prepare data
X = df.drop(columns=["Survived"])
y = df["Survived"].values

# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the neural network model
class TitanicNN(nn.Module):
    def __init__(self):
        super(TitanicNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [6]:
# Instantiate the model
model = TitanicNN()

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 100
batch_size = 32
num_samples = X_train_tensor.shape[0]

for epoch in range(epochs):
    for i in range(0, num_samples, batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.4851
Epoch [20/100], Loss: 0.4649
Epoch [30/100], Loss: 0.4495
Epoch [40/100], Loss: 0.4384
Epoch [50/100], Loss: 0.4088
Epoch [60/100], Loss: 0.3975
Epoch [70/100], Loss: 0.4095
Epoch [80/100], Loss: 0.3665
Epoch [90/100], Loss: 0.3406
Epoch [100/100], Loss: 0.3244


In [7]:
# Evaluate the model
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred = (y_pred.numpy() > 0.5).astype(int)
    accuracy = (y_pred.flatten() == y_test).mean()
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7809
